<a href="https://colab.research.google.com/github/danielegenta/Progetto-MLDL/blob/master/main_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import libraries**

In [ ]:
DATASET_ROOT = 'cifar-100-python'
import os
if not os.path.isdir(DATASET_ROOT):
    !wget https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
    !tar -xf 'cifar-100-python.tar.gz'  
    !rm -rf 'cifar-100-python.tar.gz'

import logging
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms

from PIL import Image
from tqdm import tqdm

from cifar100 import split_train_validation

from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt

%matplotlib inline

**SET ARGUMENTS**

In [ ]:
import utils

arguments = utils.get_arguments()

DEVICE = arguments['DEVICE']
NUM_CLASSES = arguments["NUM_CLASSES"] 

BATCH_SIZE = arguments["BATCH_SIZE"]        # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                                            # the batch size, learning rate should change by the same factor to have comparable results

LR = arguments["LR"]                        # The initial Learning Rate
MOMENTUM = arguments["MOMENTUM"]            # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = arguments["WEIGHT_DECAY"]    # Regularization, you can keep this at the default

NUM_EPOCHS = arguments["NUM_EPOCHS"]        # Total number of training epochs (iterations over dataset)
GAMMA = arguments["GAMMA"]                  # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = arguments["LOG_FREQUENCY"]
MILESTONES = arguments["MILESTONES"]
SEED = arguments["SEED"]

OUTPUT_PATH = "RUN1"

**Define Data Preprocessing**

In [ ]:
train_transforms, eval_transforms = utils.get_train_eval_transforms()

**Prepare Dataset**

In [ ]:
train_val_dataset = utils.get_cifar_with_seed(DATASET_ROOT, train_transforms, src='train', seed=SEED)
test_dataset = utils.get_cifar_with_seed(DATASET_ROOT, train_transforms, src='test', seed=SEED)



**Prepare Network**

In [ ]:
net, criterion, optimizer, scheduler = utils.get_resnet(LR, MOMENTUM, WEIGHT_DECAY, MILESTONES, GAMMA, resnet=32)

**Train, Test, Validation functions**

In [ ]:
def train_batch(net, train_loader, criterion, optimizer, current_step, device=DEVICE):
    
    cumulative_loss =.0
    running_corrects = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        net.train()

        optimizer.zero_grad()
        outputs = net(images)
        
        _, preds = torch.max(outputs.data, 1)
        running_corrects += torch.sum(preds == labels.data).data.item()
        
        loss = criterion(outputs, labels)
        cumulative_loss += loss.item()
        
        if current_step % LOG_FREQUENCY == 0:
                print('\t\tTrain step - Step {}, Loss {}'.format(current_step, loss.item()))

        loss.backward()
        optimizer.step()

    return cumulative_loss / len(train_loader), running_corrects, current_step

def validate(net, val_loader, criterion, optimizer, device=DEVICE):
    cumulative_loss =.0
    running_corrects = 0
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        net.eval()

        optimizer.zero_grad()
        outputs = net(images)
        
        _, preds = torch.max(outputs.data, 1)
        running_corrects += torch.sum(preds == labels.data).data.item()
        
        loss = criterion(outputs, labels)
        cumulative_loss += loss.item()

        loss.backward()
        optimizer.step()

    return cumulative_loss / len(val_loader), running_corrects

def test(net, test_loader, device=DEVICE):
    
    running_corrects = 0
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        net.eval()
        outputs = net(images)
        _, preds = torch.max(outputs.data, 1)
        running_corrects += torch.sum(preds == labels.data).data.item()

    return running_corrects


**FINE TUNING FUNCTION**

In [ ]:
def fine_tuning(net, train_val_dataset, test_dataset, criterion, optimizer, 
                scheduler, max_epoch=NUM_EPOCHS, file_path=OUTPUT_PATH, device=DEVICE):
    import math
    incremental_test = []
    train_mean_stage_accuracies = []
    val_mean_stage_accuracies = []
    test_stage_accuracies = []
    net = net.to(device)
    cudnn.benchmark
    for stage in range(10):
        print(f"STARTING FINE TUNING STAGE {stage+1}...\n\n\n")
        # Get indices
        # 4000 training, 1000 validation
        train_idx, val_idx, test_idx = utils.get_kth_batch(train_val_dataset, test_dataset, stage,
                                                                 seed=SEED, train_size=.80, get='indices')
        
        # Make test set incremental
        incremental_test.extend(test_idx)
        train_set, val_set, test_set = Subset(train_val_dataset, train_idx),\
                                               Subset(train_val_dataset, val_idx),\
                                               Subset(test_dataset, incremental_test)
        # Build data loaders
        curr_train_loader = utils.get_train_loader(train_set,batch_size=BATCH_SIZE)
        curr_val_loader = utils.get_eval_loader(val_set, batch_size=BATCH_SIZE)
        curr_test_loader = utils.get_eval_loader(test_set, batch_size=BATCH_SIZE)
        
        # Init results
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []
        min_val_loss = 0
        current_step = 0
        tolerance = 3
        for epoch in range(max_epoch):
            print(f"\tSTARTING EPOCH {epoch+1}......")
            curr_result = train_batch(net, curr_train_loader, criterion, optimizer, current_step, device)
            curr_train_loss = curr_result[0]
            curr_train_accuracy = curr_result[1] / float(len(BATCH_SIZE * len(curr_train_loader)))
            current_step = curr_result[2]
            
            train_losses.append(curr_train_loss)
            train_accuracies.append(curr_train_accuracy)
            scheduler.step()
            
            curr_val_loss, val_corrects = validate(net, curr_val_loader, criterion, optimizer, device)
            val_losses.append(curr_val_loss)
            curr_val_accuracy = val_corrects / float(len(val_set))
            val_accuracies.append(curr_val_accuracy)
            
            print(f"\t\tRESULT EPOCH {epoch+1}:")
            print(f"\t\t\tTrain Loss: {curr_train_loss} - Train Accuracy: {curr_train_accuracy}")
            print(f"\t\t\tVal Loss: {curr_val_loss} - Val Accuracy: {curr_val_accuracy}")
            
            if math.isnan(curr_val_loss):
                tolerance -= 1
            else:
                tolerance = 3
            
            if tolerance == 0:
                print(f"STAGE {stage+1} -> EARLY STOPPING")
                break
            
            if min_val_loss > curr_val_loss:
                min_val_loss = curr_val_loss
                torch.save(net, f"{file_path}_best_model.pth")
        
        net = torch.load(f"{file_path}_best_model_finetuning.pth").to(device)
        epoch_test_accuracy = test(net, curr_test_loader, device) / float(len(test_set))
        test_stage_accuracies.append(epoch_test_accuracy)
        train_mean_stage_accuracies.append(np.mean(train_accuracies))
        val_mean_stage_accuracies.append(np.mean(val_accuracies))
        
        print(f"\tResults STAGE {stage+1}:")
        print(f"\t\tTrain Mean Accuracy: {train_mean_stage_accuracies[stage]}")
        print(f"\t\tVal Mean Accuracy: {val_mean_stage_accuracies[stage]}")
        print(f"\t\tTest Accuracy: {test_stage_accuracies[stage]}")
        
    return train_mean_stage_accuracies,\
           val_mean_stage_accuracies,\
           test_stage_accuracies

**FINE TUNING START**

In [ ]:
train_accuracies,\
val_accuracies,\
test_accuracies = fine_tuning(net, train_val_dataset, test_dataset,
                              criterion, optimizer, scheduler, NUM_EPOCHS)